In [151]:
# Usuals imports 
import numpy as np
import pandas as pd 
import requests
from time import sleep
import re

## Import hostels list

In [152]:
hostels_list = pd.read_json('src/hostels_list.json', encoding='utf-8')

In [153]:
def html_unicorn(sentence):
    foo = sentence.replace("\\/", "/").encode().decode('utf-8')
    foo = foo.replace(f'\n', '')
    return foo

In [154]:
for index in range(len(hostels_list)):
    if hostels_list['hotel_name'][index] == None:
        hostel_url = hostels_list['booking_page'][index]
        h_name = re.search(r'fr/(.*?).fr', hostel_url).group(1)
        h_name = h_name.replace('-', ' ')
        
        hostels_list['hotel_name'][index] = h_name
    

In [155]:
hostels_list = hostels_list.dropna().reset_index().drop(columns=['index'])
hostels_list = hostels_list.applymap(html_unicorn)

## Create lat / long columns

In [156]:
hostels_list['latitude'] = np.nan
hostels_list['longitude'] = np.nan

## Clear adress 

In [187]:
def clear_adress(sentence):
    adress = sentence
    try :
        todrop = re.findall(r', (?:...) arr.', adress)[0]
        adress = adress.replace(todrop, '')
    except:
        pass

    try :
        commas_index = re.search(r'[0-9]+(.*?) ', adress).end()-2
        adress = adress[:commas_index] + adress[commas_index+1:]
    except:
        pass

    return adress
    

In [188]:
import requests
import urllib.parse

address = clear_adress(hostels_list['adress'][0])

url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'

response = requests.get(url).json()
print(response[0]["lat"])
print(response[0]["lon"])

48.83221
2.3163892
